In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pvlib
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
full_pv = pickle.load(open("./raw/generation.pickle", "rb"))

In [4]:
full_pv = pd.DataFrame({"Datetime":full_pv.times, "AC": full_pv.ac})

In [5]:
full_pv

,Datetime,AC
time,,
2006-01-01 00:10:00,2006-01-01 00:10:00,-3.03
2006-01-01 01:10:00,2006-01-01 01:10:00,-3.03
2006-01-01 02:10:00,2006-01-01 02:10:00,-3.03
2006-01-01 03:10:00,2006-01-01 03:10:00,-3.03
2006-01-01 04:10:00,2006-01-01 04:10:00,-3.03
...,...,...
2010-12-31 19:10:00,2010-12-31 19:10:00,-3.03
2010-12-31 20:10:00,2010-12-31 20:10:00,-3.03
2010-12-31 21:10:00,2010-12-31 21:10:00,-3.03


In [7]:
print("Max AC: ", np.max(full_pv.AC))

440.0727161894718

In [8]:
np.min(full_pv.AC)


-4.2697428328038

In [6]:
pv = full_pv[['Datetime', 'AC']].copy()
pv.loc[:,'Datetime'] = pd.to_datetime(pv.loc[:,'Datetime'])
pv.set_index('Datetime', inplace=True)
pv = pv.loc[:, ['AC']] / 1000 # W to kW
pv = pv.resample('min').interpolate(method='linear')

In [7]:
pv_episodes = pv.groupby(pd.Grouper(freq='W'))

In [8]:
pv_episodes.describe()

AC                                                            \
              count      mean       std       min      25%      50%       75%   
Datetime                                                                        
2006-01-01   1430.0  0.001731  0.011861 -0.003030 -0.00303 -0.00303 -0.003020   
2006-01-08  10080.0  0.003207  0.014177 -0.003030 -0.00303 -0.00303  0.000835   
2006-01-15  10080.0  0.001940  0.011700 -0.003030 -0.00303 -0.00303 -0.000935   
2006-01-22  10080.0  0.006655  0.019229 -0.003030 -0.00303 -0.00303  0.004989   
2006-01-29  10080.0  0.005393  0.018793 -0.003354 -0.00303 -0.00303  0.003804   
...             ...       ...       ...       ...      ...      ...       ...   
2010-12-05  10080.0  0.006405  0.018599 -0.004011 -0.00303 -0.00303  0.001588   
2010-12-12  10080.0  0.003316  0.015074 -0.003438 -0.00303 -0.00303 -0.001241   
2010-12-19  10080.0  0.002317  0.012193 -0.003030 -0.00303 -0.00303 -0.002105   
2010-12-26  10080.0  0.004858  0.016087 -0.003532 -0.00303 -0.00303  0.000619   
2011-01-02   7151.0  0.004938  0.016889 -0.003498 -0.00303 -0.00303 -0.000243   

                      
                 max  
Datetime              
2006-01-01  0.051639  
2006-01-08  0.068179  
2006-01-15  0.065029  
2006-01-22  0.079884  
2006-01-29  0.094813  
...              ...  
2010-12-05  0.073983  
2010-12-12  0.069827  
2010-12-19  0.065143  
2010-12-26  0.063219  
2011-01-02  0.067175  

[262 rows x 8 columns]

In [9]:
pv_episodes = pd.concat([group.fillna(0) for name, group in pv_episodes if len(group.dropna()) >= 10000 and group.index[0] >= datetime(2007, 1, 1) and group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))

In [10]:
pv_episodes.describe()

AC                                                            \
              count      mean       std       min      25%      50%       75%   
Datetime                                                                        
2007-01-07  10080.0  0.004354  0.016213 -0.003346 -0.00303 -0.00303 -0.001241   
2007-01-14  10080.0  0.005275  0.017286 -0.004093 -0.00303 -0.00303  0.001831   
2007-01-21  10080.0  0.005477  0.018211 -0.003030 -0.00303 -0.00303  0.000109   
2007-01-28  10080.0  0.008312  0.022100 -0.003030 -0.00303 -0.00303  0.006446   
2007-02-04  10080.0  0.006042  0.017737 -0.003030 -0.00303 -0.00303  0.009365   
...             ...       ...       ...       ...      ...      ...       ...   
2008-11-30  10080.0  0.006745  0.019220 -0.003030 -0.00303 -0.00303  0.005067   
2008-12-07  10080.0  0.003825  0.014540 -0.003485 -0.00303 -0.00303  0.001862   
2008-12-14  10080.0  0.004797  0.016509 -0.003891 -0.00303 -0.00303 -0.001425   
2008-12-21  10080.0  0.002746  0.011871 -0.003953 -0.00303 -0.00303  0.000226   
2008-12-28  10080.0  0.001406  0.011635 -0.003030 -0.00303 -0.00303 -0.002025   

                      
                 max  
Datetime              
2007-01-07  0.066303  
2007-01-14  0.067928  
2007-01-21  0.077623  
2007-01-28  0.090800  
2007-02-04  0.090245  
...              ...  
2008-11-30  0.077291  
2008-12-07  0.068325  
2008-12-14  0.066912  
2008-12-21  0.063456  
2008-12-28  0.062450  

[104 rows x 8 columns]

In [11]:
demand =  pd.HDFStore('./minutely/controlled_appliances.h5')
with pd.HDFStore('./minutely/pv.h5') as store:
    # Save each DataFrame with a key
    i = 0
    for name, group in pv_episodes:
        if name.date() == demand[f"eps_{i}"].index[-1].date():
            store[f'eps_{i}'] = group
            i += 1
file = pd.HDFStore('./minutely/generation.h5')
print(len(file) == len(demand))
file.close()
demand.close()

True
